# Section 4 : Extracting a region from a mesh

This process is considerably more involved than with "structured" data like UM.

For instance, UM data has data and coordinates with X and Y dimensions, corresponding to cell indices in the model arrays, and longitudes and latitudes of cells on the globe.  
Therefore, we can slice out a rectangular range of X and Y indices, e.g. `my_datacube[..., 10:40, 4:77]` and the result is some contiguous region of the globe within a defined range of latitude+longitude.

However, the unstructured mesh does not visit locations on the globe in any particular, simple regular pattern.  So crucially, a slice of data from the (now 1-D) arrays is not a contiguous geographical region.  And conversely a contiguous region of the data is generally not contained in a contiguous range of data indices.  
( *TODO: picture of this ?* )

So we must use geographical calculations to extract mesh data within a required region.  
Since this is a geographical concept, Geovista provides support for it.

Here's an example of how to extract the mesh falling within a defined lat-lon region ...  
**NOTE: as with the plotting example, there are no Iris utility functions for this, so a fair amount of user code is currently required to mediate between the Iris and Geovista/PyVista worlds.**

---

**First, import the utility function `lfric_rh_datacube` from `testdata_fetching`, and call it to get a global LFRic test cube.**

In [1]:
from testdata_fetching import lfric_rh_singletime_2d
lfric_rh = lfric_rh_singletime_2d()
lfric_rh

Relative Humidity At Screen Level (1),--
Shape,221184
Mesh coordinates,
latitude,x
longitude,x
Mesh,
name,Topology data of 2D unstructured mesh
location,face
Scalar coordinates,
forecast_period,21600 seconds
forecast_reference_time,2021-03-24 00:00:00


---

**Create a Polydata object from this.**  
Use the routine `pv_from_lfric_cube` from the package `pv_conversions`, which we used in the plotting section.

In [3]:
from pv_conversions import pv_from_lfric_cube
pv_global_rh = pv_from_lfric_cube(lfric_rh)

---

Now we will create a tool to extract over a desired region.

**Import the class `BBox` from `geovista.geodesic`, and make one...**  

In [4]:
from geovista.geodesic import BBox

Note: the name here is short for "Bounding Box".

**Use the notebook "?" command to display the function signature of its constructor : `?BBox.__init__`**

In [5]:
?BBox.__init__

Signature:
BBox.__init__(
    self,
    lons: Union[numpy.__array_like._SupportsArray[numpy.dtype], numpy.__nested_sequence._NestedSequence[numpy.__array_like._SupportsArray[numpy.dtype]], bool, int, float, complex, str, bytes, numpy.__nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]]],
    lats: Union[numpy.__array_like._SupportsArray[numpy.dtype], numpy.__nested_sequence._NestedSequence[numpy.__array_like._SupportsArray[numpy.dtype]], bool, int, float, complex, str, bytes, numpy.__nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]]],
    ellps: Optional[str] = 'WGS84',
    c: Optional[int] = 256,
    triangulate: Optional[bool] = False,
)
Docstring:
Create a 3D geodesic bounding-box for extracting an enclosed surface, lines
or points.

The bounding-box region is specified in terms of its four corners, in
degrees of longitude and latitude. As the bounding-box is a geodesic, it
can only ever at most enclose half of an ellipsoid.

Th

---

**Create a BBox to specify a bounding rectangle in lat-lon space.**  
Give it `lons` and `lats` arguments which specify the points of a bounding rectangle,
in lat-lon space, from 0..70 in longitude and -24..+45 in latitude.  
( *Note:* do ***not*** supply a duplicate 'end' point -- a closed loop is automatically generated. )

In [6]:
bbox = BBox(lons=[0, 70, 70, 0], lats=[-25, -25, 45, 45])

---

**Now "apply" the BBox to the global mesh data, by passing it to the `BBox.enclosed` method.**  
And show the resulting object.

In [7]:
# 'Apply' the region to the PolyData object.
pv_regional_rh = bbox.enclosed(pv_global_rh)
pv_regional_rh

PolyData (0x7fcfe604d9a0)
  N Cells:	27029
  N Points:	27380
  N Strips:	0
  X Bounds:	2.370e-01, 1.000e+00
  Y Bounds:	-8.181e-03, 9.415e-01
  Z Bounds:	-5.033e-01, 7.787e-01
  N Arrays:	5

You can see that the new (regional) polydata has fewer cells than the original (global) one.

**Now just plot this data.**  
Note : it will be useful to add coastlines as a reference to this plot.  
See 

<details><summary>Sample code solution  <b>click to reveal</b></summary>

```python
plotter = GeoVista.Plotter()
plotter.add_coastlines()
plotter.add_mesh(pv
plotter.show()
```
</details>

In [12]:
# ... space for user code ...
import geovista as gv
plotter = gv.GeoPlotter()
plotter.add_coastlines()
plotter.add_mesh(pv_regional_rh)
plotter.show()


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

**Now index the regional PolyData with `pv["vtkOriginalCellIds"]`, to get an array of cell indexes**  
and show the result.

In [32]:
# Get the remaining face indices, to use for indexing the Cube.
region_indices = pv_regional_rh["vtkOriginalCellIds"]
region_indices

pyvista_ndarray([    95,     96,     97, ..., 184179, 184180, 184181])

**Note:** This shows the original cell indices of the cells which fall within the region.

Now we can use these to select only those cell *from the Iris cube*.

**Apply these cells as an index to the 'mesh dimension' of the original Iris lfric-rh cube**  
and show that

In [52]:
lfric_rh_region = lfric_rh[..., region_indices]
lfric_rh_region

Relative Humidity At Screen Level (1),--
Shape,27029
Auxiliary coordinates,
latitude,x
longitude,x
Scalar coordinates,
forecast_period,21600 seconds
forecast_reference_time,2021-03-24 00:00:00
time,2021-03-24 06:00:00
Cell methods,
point,time


This cube contains the mesh data within our selected region.

However, there is a catch here :  Once indexed, our cube ***no longer has a mesh***.  
You can see this in the printout, which lists "Auxiliary coordinates" but no "Mesh coordinates".
This problem will probably be fixed in future.  See [here in the Iris docs](https://scitools-iris.readthedocs.io/en/latest/further_topics/ugrid/operations.html#region-extraction) for a discussion.

For now, what we need to do is to re-create a mesh for the regional cube.
We do that in a few steps ...

---

Step 1 : **Get the X and Y-axis coordinates from the region cube.**
Use `Cube.coords('longitude')` etc.

In [53]:
x_coord = lfric_rh_region.coord('longitude')
y_coord = lfric_rh_region.coord('latitude')

Step 2 : **Create a new `iris.experimental.ugrid.Mesh`-class object, passing the X,Y coords as arguments**

In [54]:
from iris.experimental.ugrid.mesh import Mesh
mesh = Mesh.from_coords(x_coord, y_coord)

( Step 2a : **`print()` the Mesh object**  
Note : `Mesh` does not provide a notebook display method.  
)

In [55]:
print(mesh)

Mesh : 'unknown'
    topology_dimension: 2
    node
        node_dimension: 'Mesh2d_node'
        node coordinates
            <AuxCoord: longitude / (degrees)  <lazy>  shape(108116,)>
            <AuxCoord: latitude / (degrees)  <lazy>  shape(108116,)>
    face
        face_dimension: 'Mesh2d_face'
        face_node_connectivity: <Connectivity: unknown / (unknown)  <lazy>  shape(27029, 4)>
        face coordinates
            <AuxCoord: longitude / (degrees)  <lazy>  shape(27029,)>
            <AuxCoord: latitude / (degrees)  <lazy>  shape(27029,)>


---
Step 3 : **call `Mesh.to_MeshCoords` to create a pair of `MeshCoord`s containing this mesh**  
Note : you must specify the keyword `location="face"` :  This matches the data location of the original data -- i.e. the data cells are faces.

In [56]:
mesh_coords = mesh.to_MeshCoords(location="face")
mesh_coords

(<MeshCoord: longitude / (degrees)  mesh(<Mesh object at 0x7f427f7821c0>) location(face)  <lazy>+bounds  shape(27029,)>,
 <MeshCoord: latitude / (degrees)  mesh(<Mesh object at 0x7f427f7821c0>) location(face)  <lazy>+bounds  shape(27029,)>)

---
Step 4 : (finally!!)  
**Use `Cube.remove_coord` and `Cube.add_aux_coord` to replace each AuxCoord with its corresponding `MeshCoord` from the previous step.** Note : for 'add_aux_coord', you also need to specify the relevant cube dimension(s) : See [`Cube.add_aux_coord` in the Iris docs](https://scitools-iris.readthedocs.io/en/latest/generated/api/iris/cube.html?highlight=add_aux_coord#iris.cube.Cube.add_aux_coord)  

And show the cube ..

In [57]:
lfric_rh_region.remove_coord('longitude')

In [58]:
lfric_rh_region.remove_coord('latitude')

In [62]:
xco, yco = mesh_coords

lfric_rh_region.add_aux_coord(xco, 0)
lfric_rh_region.add_aux_coord(yco, 0)

# Result : a regional Mesh-Cube with a subset of the original faces.
lfric_rh_region

Relative Humidity At Screen Level (1),--
Shape,27029
Mesh coordinates,
latitude,x
longitude,x
Mesh,
name,unknown
location,face
Scalar coordinates,
forecast_period,21600 seconds
forecast_reference_time,2021-03-24 00:00:00


Lastly, plot this to see what we get.

The following steps ...
  * create a PolyData from the regional cube with `pv_conversions.pv_from_lfric_cube`
  * follow the steps shown in the Plotting section

In [ ]:
pv = pv_from_lfric_cube(lfric_rh_region)
from geovista import GeoPlotter
plotter = GeoPlotter()
plotter.add_mesh(pv)
plotter.show()

----

**Investigation:** It is useful to add some extra background information to make this more visible.

As a minimum you can use `plotter.add_coastlines()`.

Another useful one is `plotter.add_base_layer()`
**What does that do ?**

In [ ]:
plotter.add_coastlines()
plotter.add_base_layer()
plotter.show()


----

**Investigation:** 
If you rotate the above images, you will see they don't behave liekt e

As a minimum you can use `plotter.add_coastlines()`.

Another useful one is `plotter.add_base_layer()`
**What does that do ?**